# Results

In this notebook we explore the results of the experiments, which are stored in wandb

In [1]:
# Get wandb api object
import wandb

api = wandb.Api()

# Get all runs
project = api.project("pysentimiento")


# Get all runs for sentiment and lang es
es_runs = [r for r in api.runs("pysentimiento", {"config.lang": "es"}) if not r.sweep]



## Spanish results

In [3]:
data = []

renames = {
    "PlanTL-GOB-ES/roberta-base-bne": "roberta",
    "bertin-project/bertin-roberta-base-spanish": "bertin",
    "dccuchile/bert-base-spanish-wwm-cased": "beto",
    "mrm8488/electricidad-base-discriminator": "electricidad",
    "pysentimiento/robertuito-base-uncased": "robertuito",
}

for run in es_runs:
    # Get model name
    model_name = run.config["model"]
    # Get task 
    task = run.config["task"]
    # Get Macro f1 performance
    macro_f1 = run.summary["test_macro_f1"]

    data.append({
        "model": model_name,
        "task": task,
        "macro_f1": macro_f1,
    })

In [5]:
import pandas as pd

df = pd.DataFrame(data)

# Group by model and task
df["model"] = df["model"].map(renames)
grouped = df.groupby(["task", "model"])

# Get mean macro f1 plus standard deviation in a single line with +- inbetween

df_results = (grouped.mean() * 100).round(1).astype(str) + " +- " + (grouped.std() * 100).round(1).astype(str)

df_results = df_results.unstack("task")
df_results.columns = df_results.columns.droplevel(0)

df_results[["sentiment", "emotion", "hate_speech", "irony"]]

task,sentiment,emotion,hate_speech,irony
model,,,,
bertin,65.3 +- 0.5,50.2 +- 2.9,68.7 +- 1.5,69.3 +- 1.4
beto,67.2 +- 0.6,52.2 +- 1.4,73.3 +- 0.8,71.5 +- 0.5
electricidad,65.3 +- 0.5,46.3 +- 2.3,71.8 +- 1.0,67.1 +- 2.1
roberta,67.3 +- 0.3,53.1 +- 2.2,73.1 +- 2.8,71.9 +- 0.9
robertuito,70.2 +- 0.2,55.3 +- 0.8,76.1 +- 0.5,74.1 +- 0.7


In [6]:
print(df_results.to_markdown())

| model        | emotion     | hate_speech   | irony       | sentiment   |
|:-------------|:------------|:--------------|:------------|:------------|
| bertin       | 50.2 +- 2.9 | 68.7 +- 1.5   | 69.3 +- 1.4 | 65.3 +- 0.5 |
| beto         | 52.2 +- 1.4 | 73.3 +- 0.8   | 71.5 +- 0.5 | 67.2 +- 0.6 |
| electricidad | 46.3 +- 2.3 | 71.8 +- 1.0   | 67.1 +- 2.1 | 65.3 +- 0.5 |
| roberta      | 53.1 +- 2.2 | 73.1 +- 2.8   | 71.9 +- 0.9 | 67.3 +- 0.3 |
| robertuito   | 55.3 +- 0.8 | 76.1 +- 0.5   | 74.1 +- 0.7 | 70.2 +- 0.2 |


In [7]:
mean_df = pd.DataFrame(data).groupby(["model", "task"]).mean()
std_df = pd.DataFrame(data).groupby(["model", "task"]).std()

# Concat mean and std
df = pd.concat([mean_df, std_df], axis=1)

df.columns = ["mean macro f1", "std macro f1"]

df = df * 100

df

mean macro f1  \
model                                      task                         
PlanTL-GOB-ES/roberta-base-bne             emotion          53.108638   
                                           hate_speech      73.076665   
                                           irony            71.900387   
                                           sentiment        67.327242   
bertin-project/bertin-roberta-base-spanish emotion          50.179562   
                                           hate_speech      68.719490   
                                           irony            69.312528   
                                           sentiment        65.261360   
dccuchile/bert-base-spanish-wwm-cased      emotion          52.157551   
                                           hate_speech      73.295649   
                                           irony            71.476460   
                                           sentiment        67.222810   
mrm8488/electricidad-base-discriminator    emotion          46.292696   
                                           hate_speech      71.832110   
                                           irony            67.068091   
                                           sentiment        65.345358   
pysentimiento/robertuito-base-uncased      emotion          55.294352   
                                           hate_speech      76.058837   
                                           irony            74.073014   
                                           sentiment        70.228989   

                                                        std macro f1  
model                                      task                       
PlanTL-GOB-ES/roberta-base-bne             emotion          2.241176  
                                           hate_speech      2.752051  
                                           irony            0.853217  
                                           sentiment        0.259381  
bertin-project/bertin-roberta-base-spanish emotion          2.937298  
                                           hate_speech      1.454827  
                                           irony            1.393909  
                                           sentiment        0.533775  
dccuchile/bert-base-spanish-wwm-cased      emotion          1.362333  
                                           hate_speech      0.848453  
                                           irony            0.547256  
                                           sentiment        0.561553  
mrm8488/electricidad-base-discriminator    emotion          2.275664  
                                           hate_speech      0.979158  
                                           irony            2.062898  
                                           sentiment        0.523014  
pysentimiento/robertuito-base-uncased      emotion          0.838796  
                                           hate_speech      0.528595  
                                           irony            0.688331  
                                           sentiment        0.150015

In [8]:
df.reset_index().to_csv("../data/results_es.csv", index=False)

In [9]:
df = pd.read_csv("../data/results_es.csv").set_index(["model", "task"])

df.loc[("pysentimiento/robertuito-base-uncased", "emotion"), "mean macro f1"]

55.2943515777588